<a href="https://colab.research.google.com/github/HodaMemar/A3/blob/main/Pivot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os.path import exists


In [4]:
filename="/content/drive/MyDrive/PaperA3/CUI/CUI__0.csv"
df=pd.read_csv(filename)
df.columns

Index(['Unnamed: 0', 'HADM_ID', 'title_span', 'category_Inner', 'negex',
       'entity_text', 'first_cuid', 'canonical_name', 'label', 'CATEGORY',
       'DRG_CODE', 'GENDER', 'AGE', 'CHARTDATE', 'TAA'],
      dtype='object')


#First solution : Integrate all CUI_#
---


برای پیوت کردن یک راه این است که همه فایل های مرحله قبل را ابتدا یکی کنیم و بعد 
وارد مرحله پیوت شویم. ولی این کار باعث کندی بسیار زیاد میشود
امکان موازی سازی هم به راحتی فراهم نیست چون عملیات روی تعدادی از رکورد های پانداس است. 


In [ ]:
filename="/content/drive/MyDrive/PaperA3/CUI/CUI__remain.csv"
df=pd.read_csv(filename)
for i in range(0,76):
  filename="/content/drive/MyDrive/PaperA3/CUI/CUI__"+str(i)+".csv"
  if (exists(filename)==True):
          print(df.shape)
          df1=pd.read_csv(filename)
          df1=df1[['Unnamed: 0', 'HADM_ID', 'title_span', 'category_Inner', 'negex',
                    'entity_text', 'first_cuid', 'canonical_name', 'label']]
          df=pd.concat([df, df1])


In [9]:
df

,Unnamed: 0,HADM_ID,title_span,category_Inner,negex,entity_text,first_cuid,canonical_name,label
0,0.0,167853.0,Allergies:,allergies,False,Flagyl,C0699678,Flagyl,CHEMICAL
1,1.0,167853.0,Chief Complaint:,chief_complaint,False,Abdominal Pain,C0000737,Abdominal Pain,DISEASE
2,2.0,167853.0,Chief Complaint:,chief_complaint,False,Heparin,C0019134,heparin,CHEMICAL
3,3.0,167853.0,Chief Complaint:,chief_complaint,False,pulmonary\nembolus,C0034065,Pulmonary Embolism,DISEASE
4,4.0,167853.0,HPI:,history_of_present_illness,False,pain,C0030193,Pain,DISEASE
...,...,...,...,...,...,...,...,...,...
102090,102090,110075,FINDINGS:,labs_and_studies,False,NSIP,C1290344,Nonspecific interstitial pneumonia,DISEASE
102091,102091,110075,FINDINGS:,labs_and_studies,True,pleural effusion,C0032227,Pleural effusion disorder,DISEASE
102092,102092,110075,FINDINGS:,labs_and_studies,True,pneumothorax,C0032326,Pneumothorax,DISEASE
102093,102093,110075,FINDINGS:,labs_and_studies,True,pneumothorax,C0032326,Pneumothorax,DISEASE


خروجی یک فایل از ترکیب همه
مفاهیم
 میشود
البته برای آمار در مقاله مهم است


---
(18075318, 10)

> Indented block




In [20]:
all_CUI=pd.read_csv('/content/drive/MyDrive/PaperA3/CUI/CUI_Total.csv')
all_CUI.shape

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,2,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(18075318, 10)

#Pivot and integrating primary dataset

این تابع برای تفاوت قائل شدن بین مفهوم مثبت و منفی است

In [4]:
def fxy(x, y):
        if(x =='True'):
            r='not ' + y 
        else:
            r=y
        return r             

یک نمونه فایل اینجا نشان داده شده است

In [5]:
filename='/content/drive/MyDrive/PaperA3/CUI/CUI__77.csv'
df=pd.read_csv(filename)

In [6]:
df.shape

(85389, 9)

In [7]:
df.head()

,Unnamed: 0,HADM_ID,title_span,category_Inner,negex,entity_text,first_cuid,canonical_name,label
0,0,167853.0,Allergies:,allergies,False,Flagyl,C0699678,Flagyl,CHEMICAL
1,1,167853.0,Chief Complaint:,chief_complaint,False,Abdominal Pain,C0000737,Abdominal Pain,DISEASE
2,2,167853.0,Chief Complaint:,chief_complaint,False,Heparin,C0019134,heparin,CHEMICAL
3,3,167853.0,Chief Complaint:,chief_complaint,False,pulmonary\nembolus,C0034065,Pulmonary Embolism,DISEASE
4,4,167853.0,HPI:,history_of_present_illness,False,pain,C0030193,Pain,DISEASE


اینجا مفاهیم منفی پردازش شده و در یک ستون جدا ذحیره میشود

In [8]:
df['Neg_CUI'] = df.apply(lambda x: fxy(x['negex'], x['first_cuid']), axis=1)

In [9]:
df.shape

(85389, 10)

In [10]:
df.columns

Index(['Unnamed: 0', 'HADM_ID', 'title_span', 'category_Inner', 'negex',
       'entity_text', 'first_cuid', 'canonical_name', 'label', 'Neg_CUI'],
      dtype='object')

In [16]:
def pivot(Pi , cat):
 tmp=df[(df['HADM_ID']==Pi ) & (df['category_Inner']==cat )]
 return [Pi,cat ,tmp.first_cuid.values.tolist()  ,tmp.canonical_name.values.tolist()  ,tmp.Neg_CUI.values.tolist()]

In [17]:
%%time

ls_patient=df.HADM_ID.unique()
          
FormName=''

ls_Category_Section=[]

for i in range(len(ls_patient)):
        try:
            ls_category=df[(df['HADM_ID']==ls_patient[i] )].category_Inner.dropna().unique().tolist()
            
            Pi=ls_patient[i]
            for j in  range(len(ls_category)):
                #FormName=df[(df['HADM_ID']==ls_patient[i] ) & (df['category_Inner']==ls_category[j] )].CATEGORY.unique()
                #print(FormName)
                cat=ls_category[j]
                ls_Category_Section.append(pivot(Pi , cat))
        except:     
          print(i,ls_patient[i])   



CPU times: user 56.7 s, sys: 134 ms, total: 56.8 s
Wall time: 56.7 s


In [18]:
ls_Category_Section[0]

[167853.0,
 'allergies',
 ['C0699678', 'C0699678'],
 ['Flagyl', 'Flagyl'],
 ['C0699678', 'C0699678']]

In [19]:
dt=pd.DataFrame(ls_Category_Section,columns=['HADM_ID','category_Inner','CUI','canonical_name','Neg_CUI'])
dt.to_csv('/content/drive/MyDrive/PaperA3/Processed/processed_77.csv')

In [21]:
ls_patient=dt.HADM_ID.unique()
print("Number of Patients:  " + str(len(ls_patient)) + ",   Number of records:  " + str(len(dt)))

Number of Patients:  2436,   Number of records:  9469
